# **Exploratory Data Analysis**:<a class="anchor" id="0"></a>


1. [**Data Import**](#1): Reading in the data

2. [**Initial Data Prep.**](#2): Run some joins and cleanup

3. [**Analysis**](#3): EDA

In [3]:
import numpy as np
import pandas as pd

from _util.custom_plotting import *
from _util.make_confusion_matrix import make_cm
from _util.model_comparisons import *
from _util.custom_mem_opt import custom_mem_opt

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
color = sns.color_palette()

root = './_data/'

## Data Import<a class=anchor id="1"></a>
[Back to top](#0)

In [5]:
products = pd.read_csv(root + 'products.csv')
print(products.head())

   product_id                                       product_name  aisle_id  \
0           1                         Chocolate Sandwich Cookies        61   
1           2                                   All-Seasons Salt       104   
2           3               Robust Golden Unsweetened Oolong Tea        94   
3           4  Smart Ones Classic Favorites Mini Rigatoni Wit...        38   
4           5                          Green Chile Anytime Sauce         5   

   department_id  
0             19  
1             13  
2              7  
3              1  
4             13  


In [6]:
aisles = pd.read_csv(root + 'aisles.csv')
print(aisles.head())

   aisle_id                       aisle
0         1       prepared soups salads
1         2           specialty cheeses
2         3         energy granola bars
3         4               instant foods
4         5  marinades meat preparation


In [7]:
departments = pd.read_csv(root + 'departments.csv')
print(departments.head())

   department_id department
0              1     frozen
1              2      other
2              3     bakery
3              4    produce
4              5    alcohol


In [8]:
orders = pd.read_csv(root + 'orders.csv')
print(orders.head())

   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2398795        1    prior             2          3                  7   
2    473747        1    prior             3          3                 12   
3   2254736        1    prior             4          4                  7   
4    431534        1    prior             5          4                 15   

   days_since_prior_order  
0                     NaN  
1                    15.0  
2                    21.0  
3                    29.0  
4                    28.0  


In [9]:
order_products_train = pd.read_csv(root + 'order_products__train.csv')
print(order_products_train.head())

   order_id  product_id  add_to_cart_order  reordered
0         1       49302                  1          1
1         1       11109                  2          1
2         1       10246                  3          0
3         1       49683                  4          0
4         1       43633                  5          1


In [10]:
order_products_prior = pd.read_csv(root + 'order_products__prior.csv')
print(order_products_prior.head())

   order_id  product_id  add_to_cart_order  reordered
0         2       33120                  1          1
1         2       28985                  2          1
2         2        9327                  3          0
3         2       45918                  4          1
4         2       30035                  5          0
